In [2]:
import requests
from lxml import html
import pandas as pd
import numpy as np

from selenium import webdriver

# Парисинг requests, lxml

In [1]:
"""
RossNeft web parsing

return: rn - pd.DataFrame(columns=['Region', 'Address', 'Brand'])
"""

main_page = 'http://www.rosneft-azs.ru/Spisok-AZS?page=1&brand=-1&region=-1&fuel=0&service=0&search='

page = requests.get(main_page).content
html_tree = html.fromstring(page, parser=html.HTMLParser(encoding='utf-8'))

address = []
region = []
brand = []

for add in html_tree.xpath("//div[@class = 'cell s-3'][2]"):
    address += [add.text]
for reg in html_tree.xpath("//div[@class = 'cell s-3'][1]"):
    region += [reg.text]
for b in html_tree.xpath("//div[@class = 'cell s-2'][1]"):
    brand += [b.text]
    
finish = int(html_tree.xpath("//div[@class = 'pagination__page'][3]//a")[0].text)
for page in range(1,finish):

    if page == 1:
        link = 'http://www.rosneft-azs.ru/Spisok-AZS' + html_tree.xpath("//div[@class = 'pagination__page'][1]//a//@href")[0]
        page = requests.get(link).content
        html_tree = html.fromstring(page, parser=html.HTMLParser(encoding='utf-8'))
        for add in html_tree.xpath("//div[@class = 'cell s-3'][2]"):
            address += [add.text]
         
        for reg in html_tree.xpath("//div[@class = 'cell s-3'][1]"):
            region += [reg.text]
       
        for b in html_tree.xpath("//div[@class = 'cell s-2'][1]"):
            brand += [b.text]
    
    else:
        if 4>page>=2:
            link = 'http://www.rosneft-azs.ru/Spisok-AZS' + html_tree.xpath("//div[@class = 'pagination__page'][{}]//a//@href".format(page))[0]
            page = requests.get(link).content
            html_tree = html.fromstring(page, parser=html.HTMLParser(encoding='utf-8'))
            for add in html_tree.xpath("//div[@class = 'cell s-3'][2]"):
                address += [add.text]
          
            for reg in html_tree.xpath("//div[@class = 'cell s-3'][1]"):
                region += [reg.text]
       
            for b in html_tree.xpath("//div[@class = 'cell s-2'][1]"):
                brand += [b.text]
     
        else:
            link = 'http://www.rosneft-azs.ru/Spisok-AZS' + html_tree.xpath("//div[@class = 'pagination__page'][4]//a//@href")[0]
            page = requests.get(link).content
            html_tree = html.fromstring(page, parser=html.HTMLParser(encoding='utf-8'))
            for add in html_tree.xpath("//div[@class = 'cell s-3'][2]"):
                address += [add.text]

            for reg in html_tree.xpath("//div[@class = 'cell s-3'][1]"):
                region += [reg.text]
        
            for b in html_tree.xpath("//div[@class = 'cell s-2'][1]"):
                brand += [b.text]
    
                
delete1 = '\n        '
delete2 = '\r\n                        '   

el=0
while el < len(region):
    if region[el] == delete1 or region[el] == delete2:
        del region[el]
    else:
        el+=1
        
el=0
while el < len(address):
    if address[el] == delete1 or address[el] == delete2:
        del address[el]
    else:
        el+=1
el=0
while el < len(brand):
    if brand[el] == delete1 or address[el] == delete2:
        del brand[el]
    else:
        el+=1
rn = pd.DataFrame(zip(region, address, brand), columns=['Region', 'Address', 'Brand'])
rn.head()

,Region,Address,Brand
0,Курганская область,"Курганская обл., г. Курган, ул. Омская, 74В",Роснефть
1,Курганская область,"Курганская обл., г. Курган, ул. Илизарова, 1 к.1",Роснефть
2,Курганская область,"Курганская обл., г. Курган, Галкинский переезд, 6",Роснефть
3,Курганская область,"Курганская обл., Белозерский р-н, с. Белозерск...",Роснефть
4,Курганская область,"Курганская обл., г. Курган, пр-т Машиностроите...",Роснефть


# Парсинг Selenium

In [3]:
"""
SNG web parsing

return: sng - pd.DataFrame(columns=['Region', 'Address', 'Brand'])
"""

addresses = []
regions = []

driver = webdriver.Chrome('C:\\chromedriver.exe')
driver.get("https://pskovnp.ru/map/")
driver.implicitly_wait(3)
elements = driver.find_elements_by_class_name("map-table--locations")

for el in elements:
    adress = el.find_element_by_tag_name('span').get_attribute('innerHTML')
    addresses.append(adress)
    regions.append('Псковская область')

driver.close()
driver.quit()

sng = pd.DataFrame(zip(addresses, regions), columns=['Address', 'Region'])
sng['Brand'] = 'СургутНефтеГаз'
sng.head()

,Address,Region,Brand
0,"Псковская обл., Псковский р-н, сельское поселе...",Псковская область,СургутНефтеГаз
1,"Псковская обл., г.Псков, ул. Вокзальная, д. 19...",Псковская область,СургутНефтеГаз
2,"Псковская обл., г.Псков, ш. Ленинградское , 6.",Псковская область,СургутНефтеГаз
3,"Псковская область, р-н Палкинский, ГП ""Палкино...",Псковская область,СургутНефтеГаз
4,"Псковская обл., Печорский р-н, СП ""Лавровская ...",Псковская область,СургутНефтеГаз
